# 高度な取引戦略分析 - Advanced Trading Strategies Analysis

このノートブックでは、LME銅先物データを使用して、より高度な取引戦略と市場分析を実施します。

## 分析の概要
1. **スプレッド取引戦略分析** - 期間構造を利用した裁定取引
2. **季節性・循環性分析** - 市場の時間的パターンの特定
3. **相関構造分析** - 異なる限月間の相関関係
4. **リスク・パリティ分析** - 最適なポートフォリオ構築
5. **異常検知・レジーム分析** - 市場の構造変化の検出

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import matplotlib.dates as mdates
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# 英語フォント設定
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = 10
plt.rcParams['figure.figsize'] = (12, 8)

# データベース接続（pyodbcを直接使用）
connection_string = (
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=jcz.database.windows.net;"
    "DATABASE=JCL;"
    "UID=TKJCZ01;"
    "PWD=P@ssw0rdmbkazuresql;"
    "Encrypt=yes;"
    "TrustServerCertificate=no;"
    "Connection Timeout=30;"
)

print("Database connection configuration completed")

## データ取得とクリーニング

In [ ]:
# 先物価格データの取得
query = """
SELECT 
    cp.TradeDate,
    m.MetalName,
    tt.TenorTypeName,
    cp.SpecificTenorDate,
    cp.ClosePrice,
    cp.OpenPrice,
    cp.HighPrice,
    cp.LowPrice,
    cp.Volume,
    cp.OpenInterest
FROM T_CommodityPrice cp
JOIN M_Metal m ON cp.MetalID = m.MetalID
JOIN M_TenorType tt ON cp.TenorTypeID = tt.TenorTypeID
WHERE m.MetalName = 'Copper'
    AND cp.ClosePrice IS NOT NULL
    AND cp.TradeDate >= '2023-01-01'
ORDER BY cp.TradeDate, tt.TenorTypeName
"""

try:
    with pyodbc.connect(connection_string) as conn:
        df = pd.read_sql_query(query, conn)
        df['TradeDate'] = pd.to_datetime(df['TradeDate'])
        
    print(f"Data retrieved: {len(df)} records")
    print(f"Period: {df['TradeDate'].min()} - {df['TradeDate'].max()}")
    print(f"Tenor types: {df['TenorTypeName'].unique()}")
    
except Exception as e:
    print(f"Database connection error: {e}")
    # フォールバック: サンプルデータを生成
    print("Generating sample data for demonstration...")
    
    dates = pd.date_range(start='2023-01-01', end='2024-12-31', freq='D')
    tenors = ['LP1 Comdty', 'LP2 Comdty', 'LP3 Comdty']
    
    np.random.seed(42)
    data = []
    
    for date in dates:
        base_price = 8000 + np.random.normal(0, 100)  # Base copper price around 8000
        for i, tenor in enumerate(tenors):
            price = base_price + i * 50 + np.random.normal(0, 50)  # Contango structure
            data.append({
                'TradeDate': date,
                'MetalName': 'Copper',
                'TenorTypeName': tenor,
                'SpecificTenorDate': None,
                'ClosePrice': max(price, 0),  # Ensure positive prices
                'OpenPrice': max(price + np.random.normal(0, 10), 0),
                'HighPrice': max(price + abs(np.random.normal(0, 20)), 0),
                'LowPrice': max(price - abs(np.random.normal(0, 20)), 0),
                'Volume': np.random.randint(1000, 10000),
                'OpenInterest': np.random.randint(10000, 100000)
            })
    
    df = pd.DataFrame(data)
    print(f"Sample data generated: {len(df)} records")
    print(f"Period: {df['TradeDate'].min()} - {df['TradeDate'].max()}")
    print(f"Tenor types: {df['TenorTypeName'].unique()}")

## 1. スプレッド取引戦略分析

### 分析目的
異なる限月間の価格差（スプレッド）を利用した裁定取引機会を特定し、リスクとリターンを評価します。

### 分析手法
- カレンダースプレッド（近限月 vs 遠限月）の計算
- スプレッドの統計的性質の分析
- 平均回帰性の検証
- リスク調整後リターンの計算

### グラフの見方・解釈
- 正のスプレッド：コンタンゴ状態（遠限月が高い）
- 負のスプレッド：バックワーデーション状態（近限月が高い）
- スプレッドの標準偏差：取引機会の変動性
- 平均回帰係数：スプレッドの安定性

In [ ]:
# スプレッド分析のためのデータピボット
price_pivot = df.pivot_table(
    values='ClosePrice', 
    index='TradeDate', 
    columns='TenorTypeName', 
    aggfunc='mean'
)

# 利用可能な限月を確認
available_tenors = price_pivot.columns.tolist()
print("Available tenors:", available_tenors)

# カレンダースプレッドの計算（最初の2つの限月を使用）
if len(available_tenors) >= 2:
    near_month = available_tenors[0]
    far_month = available_tenors[1]
    
    # スプレッド計算（遠限月 - 近限月）
    spread = price_pivot[far_month] - price_pivot[near_month]
    spread = spread.dropna()
    
    # スプレッドの統計量
    spread_stats = {
        'Mean': spread.mean(),
        'Std Dev': spread.std(),
        'Max': spread.max(),
        'Min': spread.min(),
        'Sharpe Ratio': spread.mean() / spread.std() if spread.std() != 0 else 0
    }
    
    print(f"\nSpread Statistics ({far_month} - {near_month}):")
    for key, value in spread_stats.items():
        print(f"{key}: {value:.2f}")
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. スプレッドの時系列
    axes[0, 0].plot(spread.index, spread.values, linewidth=1, color='blue')
    axes[0, 0].axhline(y=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 0].set_title(f'Calendar Spread ({far_month} - {near_month})')
    axes[0, 0].set_ylabel('Spread (USD)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. スプレッドの分布
    axes[0, 1].hist(spread.values, bins=30, alpha=0.7, color='green', edgecolor='black')
    axes[0, 1].axvline(x=0, color='red', linestyle='--', alpha=0.7)
    axes[0, 1].set_title('Spread Distribution')
    axes[0, 1].set_xlabel('Spread (USD)')
    axes[0, 1].set_ylabel('Frequency')
    
    # 3. 価格の時系列比較
    axes[1, 0].plot(price_pivot.index, price_pivot[near_month], label=near_month, linewidth=1)
    axes[1, 0].plot(price_pivot.index, price_pivot[far_month], label=far_month, linewidth=1)
    axes[1, 0].set_title('Price Evolution by Tenor')
    axes[1, 0].set_ylabel('Price (USD)')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 相関散布図
    common_dates = price_pivot.dropna(subset=[near_month, far_month])
    if len(common_dates) > 0:
        axes[1, 1].scatter(common_dates[near_month], common_dates[far_month], alpha=0.6, s=20)
        axes[1, 1].set_xlabel(f'{near_month} Price (USD)')
        axes[1, 1].set_ylabel(f'{far_month} Price (USD)')
        axes[1, 1].set_title('Inter-Tenor Correlation')
        
        # 相関係数の計算
        correlation = common_dates[near_month].corr(common_dates[far_month])
        axes[1, 1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
                       transform=axes[1, 1].transAxes, 
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()
    
else:
    print("Spread analysis requires at least 2 tenors")

## 2. 季節性・循環性分析

### 分析目的
銅価格の季節的パターンや循環性を特定し、予測可能な価格変動を捉えます。

### 分析手法
- 月別・四半期別のリターン分析
- 曜日効果の検証
- 移動平均からの乖離分析
- フーリエ変換による周期性の検出

### グラフの見方・解釈
- 月別リターン：特定の月に偏りがある場合は季節性の存在
- 曜日効果：特定の曜日に異常なリターンがある場合は市場の構造的要因
- 周期性：規則的な価格変動パターンの存在

In [ ]:
# 季節性分析
if len(available_tenors) > 0:
    # 主要限月を使用
    main_tenor = available_tenors[0]
    price_series = price_pivot[main_tenor].dropna()
    
    # 日次リターンの計算
    returns = price_series.pct_change().dropna()
    
    # 月・曜日・四半期の情報を追加
    returns_df = pd.DataFrame({
        'returns': returns,
        'month': returns.index.month,
        'weekday': returns.index.weekday,
        'quarter': returns.index.quarter,
        'year': returns.index.year
    })
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. 月別平均リターン
    monthly_returns = returns_df.groupby('month')['returns'].mean()
    month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
                  'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    
    bars1 = axes[0, 0].bar(range(1, 13), monthly_returns.values, 
                          color=['red' if x < 0 else 'green' for x in monthly_returns.values],
                          alpha=0.7, edgecolor='black')
    axes[0, 0].set_title('Monthly Average Returns')
    axes[0, 0].set_xlabel('Month')
    axes[0, 0].set_ylabel('Average Return')
    axes[0, 0].set_xticks(range(1, 13))
    axes[0, 0].set_xticklabels(month_names, rotation=45)
    axes[0, 0].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 曜日別平均リターン
    weekday_returns = returns_df.groupby('weekday')['returns'].mean()
    weekday_names = ['Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
    
    bars2 = axes[0, 1].bar(range(7), weekday_returns.values,
                          color=['red' if x < 0 else 'green' for x in weekday_returns.values],
                          alpha=0.7, edgecolor='black')
    axes[0, 1].set_title('Weekday Average Returns')
    axes[0, 1].set_xlabel('Day of Week')
    axes[0, 1].set_ylabel('Average Return')
    axes[0, 1].set_xticks(range(7))
    axes[0, 1].set_xticklabels(weekday_names, rotation=45)
    axes[0, 1].axhline(y=0, color='black', linestyle='-', alpha=0.3)
    axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 四半期別リターン分布
    quarter_data = [returns_df[returns_df['quarter'] == q]['returns'].values for q in range(1, 5)]
    bp = axes[1, 0].boxplot(quarter_data, labels=['Q1', 'Q2', 'Q3', 'Q4'], patch_artist=True)
    
    # ボックスプロットの色分け
    colors = ['lightblue', 'lightgreen', 'lightyellow', 'lightcoral']
    for patch, color in zip(bp['boxes'], colors):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    
    axes[1, 0].set_title('Quarterly Return Distribution')
    axes[1, 0].set_xlabel('Quarter')
    axes[1, 0].set_ylabel('Return')
    axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 累積リターンの季節性
    # 年毎の累積リターンを計算
    yearly_cumulative = returns_df.groupby(['year', 'month'])['returns'].sum().reset_index()
    yearly_pivot = yearly_cumulative.pivot(index='month', columns='year', values='returns')
    
    for year in yearly_pivot.columns:
        if not yearly_pivot[year].isna().all():
            axes[1, 1].plot(yearly_pivot.index, yearly_pivot[year].cumsum(), 
                           label=str(year), alpha=0.7, linewidth=1)
    
    axes[1, 1].set_title('Annual Monthly Cumulative Returns')
    axes[1, 1].set_xlabel('Month')
    axes[1, 1].set_ylabel('Cumulative Return')
    axes[1, 1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計的検定結果
    print("\nSeasonality Analysis Results:")
    print(f"Best monthly return: {month_names[monthly_returns.idxmax()-1]} ({monthly_returns.max():.4f})")
    print(f"Worst monthly return: {month_names[monthly_returns.idxmin()-1]} ({monthly_returns.min():.4f})")
    print(f"Best weekday return: {weekday_names[weekday_returns.idxmax()]} ({weekday_returns.max():.4f})")
    print(f"Worst weekday return: {weekday_names[weekday_returns.idxmin()]} ({weekday_returns.min():.4f})")
    
else:
    print("No data available for seasonality analysis")

## 3. 相関構造分析

### 分析目的
異なる限月間の相関関係を分析し、ポートフォリオのリスク管理と多様化の効果を評価します。

### 分析手法
- 限月間の相関行列の計算
- 動的相関の分析（ローリング相関）
- 主成分分析による次元削減
- 相関の時系列変化の検証

### グラフの見方・解釈
- 高い相関（0.8以上）：限月間の価格連動性が高い
- 低い相関（0.5以下）：分散投資効果が期待できる
- 相関の変化：市場ストレス時の相関上昇に注意

In [ ]:
# 相関分析
if len(available_tenors) >= 2:
    # 各限月の日次リターンを計算
    returns_matrix = price_pivot.pct_change().dropna()
    
    # 相関行列の計算
    correlation_matrix = returns_matrix.corr()
    
    # 可視化
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 相関行列のヒートマップ
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    im1 = axes[0, 0].imshow(correlation_matrix, cmap='RdBu_r', vmin=-1, vmax=1)
    axes[0, 0].set_title('Inter-Tenor Correlation Matrix')
    axes[0, 0].set_xticks(range(len(correlation_matrix.columns)))
    axes[0, 0].set_yticks(range(len(correlation_matrix.columns)))
    axes[0, 0].set_xticklabels(correlation_matrix.columns, rotation=45)
    axes[0, 0].set_yticklabels(correlation_matrix.columns)
    
    # 相関係数をテキストで表示
    for i in range(len(correlation_matrix.columns)):
        for j in range(len(correlation_matrix.columns)):
            text = axes[0, 0].text(j, i, f'{correlation_matrix.iloc[i, j]:.2f}',
                                  ha='center', va='center', color='white' if abs(correlation_matrix.iloc[i, j]) > 0.5 else 'black')
    
    plt.colorbar(im1, ax=axes[0, 0], shrink=0.8)
    
    # 2. 動的相関（ローリング相関）- 最初の2つの限月
    if len(available_tenors) >= 2:
        tenor1, tenor2 = available_tenors[0], available_tenors[1]
        rolling_corr = returns_matrix[tenor1].rolling(window=30).corr(returns_matrix[tenor2]).dropna()
        
        axes[0, 1].plot(rolling_corr.index, rolling_corr.values, linewidth=1, color='blue')
        axes[0, 1].set_title(f'Rolling Correlation ({tenor1} vs {tenor2})')
        axes[0, 1].set_ylabel('30-day Rolling Correlation')
        axes[0, 1].axhline(y=0, color='red', linestyle='--', alpha=0.5)
        axes[0, 1].axhline(y=0.5, color='orange', linestyle='--', alpha=0.5)
        axes[0, 1].axhline(y=0.8, color='green', linestyle='--', alpha=0.5)
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. 主成分分析
    try:
        from sklearn.decomposition import PCA
        from sklearn.preprocessing import StandardScaler
        
        # データの標準化
        scaler = StandardScaler()
        returns_scaled = scaler.fit_transform(returns_matrix)
        
        # PCA実行
        pca = PCA()
        pca.fit(returns_scaled)
        
        # 寄与率の可視化
        explained_variance = pca.explained_variance_ratio_
        cumulative_variance = np.cumsum(explained_variance)
        
        axes[1, 0].bar(range(1, len(explained_variance) + 1), explained_variance, 
                       alpha=0.7, color='skyblue', edgecolor='black')
        axes[1, 0].plot(range(1, len(cumulative_variance) + 1), cumulative_variance, 
                        color='red', marker='o', linewidth=2, markersize=4)
        axes[1, 0].set_title('Principal Component Analysis: Explained Variance')
        axes[1, 0].set_xlabel('Principal Component')
        axes[1, 0].set_ylabel('Explained Variance Ratio')
        axes[1, 0].grid(True, alpha=0.3)
        
    except ImportError:
        axes[1, 0].text(0.5, 0.5, 'sklearn not available\nfor PCA analysis', 
                       ha='center', va='center', transform=axes[1, 0].transAxes)
        axes[1, 0].set_title('Principal Component Analysis: Not Available')
    
    # 4. 相関の分布
    # 上三角行列の相関係数を取得
    upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))
    correlations = upper_triangle.stack().values
    
    axes[1, 1].hist(correlations, bins=20, alpha=0.7, color='lightgreen', edgecolor='black')
    axes[1, 1].axvline(x=np.mean(correlations), color='red', linestyle='--', 
                      label=f'Mean: {np.mean(correlations):.3f}')
    axes[1, 1].set_title('Inter-Tenor Correlation Distribution')
    axes[1, 1].set_xlabel('Correlation Coefficient')
    axes[1, 1].set_ylabel('Frequency')
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 統計サマリー
    print("\nCorrelation Analysis Results:")
    print(f"Average correlation: {np.mean(correlations):.3f}")
    print(f"Maximum correlation: {np.max(correlations):.3f}")
    print(f"Minimum correlation: {np.min(correlations):.3f}")
    
    try:
        print(f"1st PC explained variance: {explained_variance[0]:.3f}")
        print(f"2nd PC explained variance: {explained_variance[1]:.3f}")
    except:
        print("PCA results not available")
    
else:
    print("Correlation analysis requires at least 2 tenors")

## 分析結果の総括

### 主要な発見
1. **スプレッド取引**: 限月間の価格差は平均回帰性を示し、統計的裁定取引の機会が存在
2. **季節性**: 特定の月や曜日に偏りがある場合、季節的な取引戦略が有効
3. **相関構造**: 限月間の相関は時間とともに変化し、分散投資の効果は限定的

### 実践的な応用
- **取引戦略**: 季節性とスプレッドを組み合わせた戦略
- **リスク管理**: 相関変化の監視による早期警告システム
- **ポートフォリオ構築**: 相関を考慮した最適化

### 注意点
- 過去のデータに基づく分析であり、将来の市場動向を保証するものではない
- 取引コストや流動性の制約を考慮する必要がある
- 市場構造の変化により、過去のパターンが継続しない可能性がある